In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import PIL.Image as Image

Get data

In [2]:
IMAGE_SIZE = (28, 28)
EPOCH = 3
# When weights become NaN, try reducing the learning rate
LEARNING_RATE = 0.00001

# Import data
data = pd.read_csv(r"../digit-recognizer/train.csv")
data_test = pd.read_csv(r"../digit-recognizer/test.csv")    

# Get labels and image array from data
labels : np.ndarray = data.values[:, 0]
images : np.ndarray = data.values[:, 1:].astype('uint8')

images_test : np.ndarray = data_test.values.astype('uint8')

Process data

# Test sofmax

In [6]:
class ModelSoftmax(nn.Module):
    def __init__(self, input, hidden, output):
        super().__init__()
        self.layer1 = nn.Linear(input, hidden)
        self.layer2 = nn.Linear(hidden, output)

        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        # torch.sigmoid(x)

        return x

In [9]:
model = ModelSoftmax(IMAGE_SIZE[0] * IMAGE_SIZE[1], 512, 10)
# Cross Entropy Loss already includes softmax
criterion = nn.CrossEntropyLoss()   
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)
for i in range(EPOCH):
    wrong = 0
    for j in range(len(images)):
        optimizer.zero_grad()
        res = model(torch.tensor(images[j] / 255.0, dtype=torch.float32))
        if (int(torch.argmax(res)) != labels[j]):
            wrong += 1

        # Create label for calculating loss, for sigmoid
        # label_tensor = np.zeros(10)
        # label_tensor[labels[j]] = 1
        # label_tensor = torch.tensor(label_tensor, dtype=torch.float32)
        # print(label_tensor)

        # CrossEntropyLoss, target tensor only contains one label output
        label_tensor = torch.tensor([labels[j]], dtype=torch.long)
        res = torch.reshape(res, (1, 10))
        loss = criterion(res, label_tensor)
        loss.backward()
        optimizer.step()
        
        # print(model.layer1.weight)
        if ((j+1) % 2000 == 0):
            print(f"Data {j+1}: Wrong = {wrong}, Accuracy: {100-wrong/j*100}%")
         
    print(f"Epoch: {i} --> Wrong: {wrong}, Accuracy: {100-wrong / labels.size * 100}%\n")    

# Test model
wrong = 0
for i in range(100):
    res = torch.argmax(model(torch.tensor(images_test[i], dtype=torch.float32)))
    img = Image.fromarray(images_test[i].reshape(IMAGE_SIZE))
    print(f"Image {i+1}: {res}")
    img.save(f"./test_softmax/{str(i+1)}_ans={res}.jpg")


Data 2000: Wrong = 1820, Accuracy: 8.954477238619305%
Data 4000: Wrong = 3573, Accuracy: 10.652663165791438%
Data 6000: Wrong = 5276, Accuracy: 12.052008668111341%
Data 8000: Wrong = 6921, Accuracy: 13.476684585573196%
Data 10000: Wrong = 8531, Accuracy: 14.68146814681468%
Data 12000: Wrong = 10066, Accuracy: 16.1096758063172%
Data 14000: Wrong = 11544, Accuracy: 17.53696692620902%
Data 16000: Wrong = 12973, Accuracy: 18.91368210513157%
Data 18000: Wrong = 14320, Accuracy: 20.440024445802536%
Data 20000: Wrong = 15604, Accuracy: 21.97609880494025%
Data 22000: Wrong = 16868, Accuracy: 23.323787444883862%
Data 24000: Wrong = 18022, Accuracy: 24.90520438351598%
Data 26000: Wrong = 19140, Accuracy: 26.38178391476596%
Data 28000: Wrong = 20261, Accuracy: 27.636701310761097%
Data 30000: Wrong = 21269, Accuracy: 29.100970032334402%
Data 32000: Wrong = 22293, Accuracy: 30.332197881183788%
Data 34000: Wrong = 23258, Accuracy: 31.592105650166175%
Data 36000: Wrong = 24182, Accuracy: 32.825911830

# Test sigmoid

In [7]:
class ModelSigmoid(nn.Module):
    def __init__(self, input, hidden, output):
        super().__init__()
        self.layer1 = nn.Linear(input, output)
        # self.layer2 = nn.Linear(hidden, output)

    def forward(self, x):
        x = self.layer1(x)
        # x = self.layer2(x)
        torch.sigmoid(x)

        return x

In [5]:
model = ModelSigmoid(IMAGE_SIZE[0] * IMAGE_SIZE[1], 1024, 10)
criterion = nn.MSELoss()   
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)
for i in range(EPOCH):
    wrong = 0
    for j in range(len(images)):
        optimizer.zero_grad()
        res = model(torch.tensor(images[j] / 255.0, dtype=torch.float32))
        if (int(torch.argmax(res)) != labels[j]):
            wrong += 1

        # Create label for calculating loss, for sigmoid
        label_tensor = np.zeros(10)
        label_tensor[labels[j]] = 1
        label_tensor = torch.tensor(label_tensor, dtype=torch.float32)
        
        loss = criterion(res, label_tensor)
        loss.backward()
        optimizer.step()
        
        # print(model.layer1.weight)
        if ((j+1) % 2000 == 0):
            print(f"Data {j+1}: Wrong = {wrong}, Accuracy: {100-wrong/j*100}%")
        
    print(f"Epoch: {i} --> Wrong: {wrong}, Accuracy: {100-wrong / labels.size * 100}%\n")    

# Test model
wrong = 0
for i in range(100):
    res = torch.argmax(model(torch.tensor(images_test[i], dtype=torch.float32)))
    img = Image.fromarray(images_test[i].reshape(IMAGE_SIZE))
    print(f"Image {i+1}: {res}")
    img.save(f"./test_sigmoid_torch/{str(i+1)}_ans={res}.jpg")

NameError: name 'ModelSigmoid' is not defined

In [13]:
testing = np.array([
    [1, 1, 1],
    [1, 0, 1],
    [0, 1, 1],
    [0, 0, 1],
    [0, 0, 0],
    [1, 0, 0],
])

label = np.array(
    [1, 1, 0, 0, 0, 1, ]
)

model = ModelSigmoid(3, 1024, 1)
criterion = nn.MSELoss()   
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)

for e in range(100000):
    results = []
    for i, d in enumerate(testing):
        optimizer.zero_grad()
        res = model(torch.tensor(d, dtype=torch.float32))
        results.append(float(res))

        loss = criterion(res, torch.tensor(label[i], dtype=torch.float32))
        loss.backward()
        optimizer.step()

    if ((e+1) % 1000 == 0):
        print(f"Epoch {e+1}: {results}")

print(model.forward(torch.tensor([1, 1, 0], dtype=torch.float32)))

Epoch 1000: [0.23741397261619568, -0.10381779819726944, 0.6830536127090454, 0.34173351526260376, 0.007772947661578655, -0.4377395808696747]
Epoch 2000: [0.35356682538986206, 0.012509912252426147, 0.7365738153457642, 0.3954356908798218, 0.048184577375650406, -0.33471235632896423]
Epoch 3000: [0.446034699678421, 0.108118437230587, 0.7725082635879517, 0.4345168173313141, 0.07997233420610428, -0.2464049905538559]
Epoch 4000: [0.5197558999061584, 0.18721237778663635, 0.7947368621826172, 0.46212321519851685, 0.10491905361413956, -0.16997689008712769]
Epoch 5000: [0.578639805316925, 0.2531244456768036, 0.8063246011734009, 0.48074325919151306, 0.12443695217370987, -0.10317178815603256]
Epoch 6000: [0.6257858276367188, 0.30849796533584595, 0.8096954822540283, 0.49234533309936523, 0.13964366912841797, -0.04419738054275513]
Epoch 7000: [0.6636345982551575, 0.3554236888885498, 0.8067500591278076, 0.4984799325466156, 0.15142585337162018, 0.008373022079467773]
Epoch 8000: [0.6941195726394653, 0.3955